In [25]:
# Helpful resource: http://andrewgaidus.com/leaflet_webmaps_python/
import pandas as pd
import geopandas as gpd
import folium
import pysal as ps
import os

project_dir = os.path.join(os.getcwd(), os.pardir)

In [12]:
df = pd.read_pickle(os.path.join(project_dir, 'data', 'joint_data_by_census.p'))
df.head()

,index,tract_id_proper,ci_score,op_score,so,srb,shs,chc,County FIPS Code,tract,tract_id
0,0,1,3.80,3.398696,0.167878,0.390522,0.353304,0.005217,107,42107000100,000100
1,3,10,4.25,3.318182,0.140718,0.396273,0.480827,0.004545,49,42049001000,001000
2,4,10.01,1.00,5.250000,0.133150,0.352400,0.435550,0.005000,101,42101001001,001001
3,5,10.02,1.50,5.000000,0.137933,0.394933,0.309267,0.003333,101,42101001002,001002
4,6,100,3.00,3.250000,0.117500,0.398500,0.573250,0.005000,101,42101010000,010000


In [16]:
pa_shape = gpd.read_file(os.path.join(project_dir, 'data', 'cb_2016_42_tract_500k.geojson'))

In [17]:
pa_shape.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,42,101,010200,1400000US42101010200,42101010200,102,CT,360618,0,"POLYGON ((-75.235356 39.968509, -75.235697 39...."
1,42,101,011900,1400000US42101011900,42101011900,119,CT,690903,0,"POLYGON ((-75.233672 39.991882, -75.225662 39...."
2,42,101,013900,1400000US42101013900,42101013900,139,CT,562937,0,"POLYGON ((-75.177852 39.974247, -75.173785 39...."
3,42,101,015700,1400000US42101015700,42101015700,157,CT,485072,0,"POLYGON ((-75.138768 39.979323, -75.138137 39...."
4,42,101,016300,1400000US42101016300,42101016300,163,CT,572226,0,"POLYGON ((-75.139021 39.988764, -75.138352 39...."


In [24]:
gdf = pa_shape.merge(df, left_on='TRACTCE', right_on='tract_id')
gdf.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,tract_id_proper,ci_score,op_score,so,srb,shs,chc,County FIPS Code,tract,tract_id
0,42,101,010200,1400000US42101010200,42101010200,102,CT,360618,0,"POLYGON ((-75.235356 39.968509, -75.235697 39....",...,102,2.25,3.504444,0.172978,0.317072,0.235817,0.001667,81,42081010200,010200
1,42,027,010200,1400000US42027010200,42027010200,102,CT,451226567,2062239,"POLYGON ((-78.112239 41.114217, -78.104866 41....",...,102,2.25,3.504444,0.172978,0.317072,0.235817,0.001667,81,42081010200,010200
2,42,095,010200,1400000US42095010200,42095010200,102,CT,7307701,81905,"POLYGON ((-75.396869 40.644551, -75.387272 40....",...,102,2.25,3.504444,0.172978,0.317072,0.235817,0.001667,81,42081010200,010200
3,42,081,010200,1400000US42081010200,42081010200,102,CT,626695968,5634589,"POLYGON ((-77.599278 41.542271, -77.454835 41....",...,102,2.25,3.504444,0.172978,0.317072,0.235817,0.001667,81,42081010200,010200
4,42,021,010200,1400000US42021010200,42021010200,102,CT,2129479,55811,"POLYGON ((-78.89387600000001 40.338569, -78.89...",...,102,2.25,3.504444,0.172978,0.317072,0.235817,0.001667,81,42081010200,010200


In [67]:
pa_map = folium.Map([41.203323, -77.194527], zoom_start = 7)
pa_map

In [68]:
gdf_wgs84 = gdf.to_crs(epsg=4326)

In [71]:
def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
#     gdf_wgs84 = gdf.to_crs(epsg=4326)
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    layer = folium.FeatureGroup(name=value_field)
    layer.add_child(mapobj.choropleth(geo_str = gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale), value_field)
    mapobj.add_children(layer)
    return mapobj

In [72]:
#Update basemap with choropleth
pa_map=add_choropleth(pa_map, gdf_wgs84, 'TRACTCE','ci_score')

AttributeError: 'NoneType' object has no attribute '_parent'

In [47]:
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join([row[field] for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(IFrame(label, width = 300, height = 100))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_children(pt_lyr)
    return mapobj

#Update choropleth with point clusters
pa_map = add_point_clusters(pa_map, gdf, ['ci_score'])

AttributeError: 'Polygon' object has no attribute 'y'

In [56]:
folium.LayerControl().add_to(pa_map) #Add layer control to toggle on/off

In [58]:
pa_map.save(os.path.join(project_dir, 'data', 'output.html'))

AssertionError: You cannot render this Element if it's not in a Figure.

In [ ]:
gdf.to_file(os.path.join(project_dir, 'data', 'output.geojson'))
gdf.to_file(os.path.join(project_dir, 'data', 'output.cs

In [84]:
gdf.to_file(os.path.join(project_dir, 'data', 'output.dgn'), driver='DGN')

DriverIOError: No seed file provided, and unable to find seed_2d.dgn.